<a href="https://colab.research.google.com/github/octavioeac/ads-spend/blob/feature%2FAdd-the-notebook-colab/AD_Spend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> **The analysis aims to clean and validate the ads dataset, calculate acquisition efficiency (CAC) and return efficiency (ROAS), and compare performance over time and across segments (platform, country, device). The goal is to identify patterns and trends that can guide marketing budget allocation and campaign optimization.**


In [3]:
url = "https://raw.githubusercontent.com/octavioeac/ads-spend/refs/heads/main/notebooks/data/ads_spend.csv"
df = pd.read_csv(url)
df.head()

,date,platform,account,campaign,country,device,spend,clicks,impressions,conversions
0,2025-01-01,Meta,AcctA,Prospecting,MX,Desktop,1115.94,360,15840,29
1,2025-01-01,Google,AcctA,Brand_Search,CA,Mobile,789.43,566,22640,28
2,2025-01-01,Google,AcctA,Prospecting,BR,Desktop,381.40,133,10241,12
3,2025-01-01,Google,AcctC,Prospecting,US,Desktop,1268.34,891,49005,36
4,2025-01-01,Google,AcctA,Brand_Search,BR,Desktop,1229.70,628,21352,31


In [4]:

# Shape of the dataset
print("Shape:", df.shape)

# 1. Null values
print("\nMissing values per column:")
print(df.isnull().sum())

# 2. Duplicates
print("\nNumber of duplicate rows:", df.duplicated().sum())

# 3. Data types (categorical vs numerical)
print("\nData types:")
print(df.dtypes)

# 4. Unique values in categorical columns
categorical_cols = df.select_dtypes(include=["object"]).columns
print("\nUnique values per categorical column:")
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} unique values")

Shape: (2000, 10)

Missing values per column:
date           0
platform       0
account        0
campaign       0
country        0
device         0
spend          0
clicks         0
impressions    0
conversions    0
dtype: int64

Number of duplicate rows: 0

Data types:
date            object
platform        object
account         object
campaign        object
country         object
device          object
spend          float64
clicks           int64
impressions      int64
conversions      int64
dtype: object

Unique values per categorical column:
date: 181 unique values
platform: 2 unique values
account: 3 unique values
campaign: 4 unique values
country: 4 unique values
device: 2 unique values


#Create a function to clean the dataset

In [5]:
def clean_dataset(df):
    # 1. Drop duplicates
  df = df.drop_duplicates()
  print("After removing duplicates:", df.shape)

  # 2. Handle missing values
  # Strategy: drop rows if key fields (date, platform, campaign, spend, conversions) are missing
  df = df.dropna(subset=["date", "platform", "campaign", "spend", "conversions"])
  print("After dropping critical nulls:", df.shape)

  # 3. Fill less critical categorical nulls with "unknown"
  for col in ["country", "device", "account"]:
      if col in df.columns:
          df[col] = df[col].fillna("unknown")

  # 4. Ensure numeric columns have valid values
  numeric_cols = ["spend", "clicks", "impressions", "conversions"]
  for col in numeric_cols:
      df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

  # 5. Protect against division by zero
  df = df[(df["conversions"] > 0) & (df["spend"] > 0)]

  # 6. Add KPIs safely
  df["revenue"] = df["conversions"] * 100
  df["cac"] = df["spend"] / df["conversions"]
  df["roas"] = df["revenue"] / df["spend"]

  return df.head()

df_clean = clean_dataset(df)
df_clean.head()


After removing duplicates: (2000, 10)
After dropping critical nulls: (2000, 10)


,date,platform,account,campaign,country,device,spend,clicks,impressions,conversions,revenue,cac,roas
0,2025-01-01,Meta,AcctA,Prospecting,MX,Desktop,1115.94,360,15840,29,2900,38.480690,2.598706
1,2025-01-01,Google,AcctA,Brand_Search,CA,Mobile,789.43,566,22640,28,2800,28.193929,3.546863
2,2025-01-01,Google,AcctA,Prospecting,BR,Desktop,381.40,133,10241,12,1200,31.783333,3.146303
3,2025-01-01,Google,AcctC,Prospecting,US,Desktop,1268.34,891,49005,36,3600,35.231667,2.838356
4,2025-01-01,Google,AcctA,Brand_Search,BR,Desktop,1229.70,628,21352,31,3100,39.667742,2.520940
